In [1]:
%env QIBO_ENVIRONMENT=local

env: QIBO_ENVIRONMENT=local


In [2]:
from qiboconnection.connection import ConnectionConfiguration
from qiboconnection.api import API
from qiboconnection.typings.algorithm import (
    AlgorithmType, InitialValue, ProgramDefinition, AlgorithmDefinition, AlgorithmOptions, AlgorithmName)


[qibo-connection] 0.0.1|DEBUG|2021-12-13 14:52:21]: Qibo Quantum Service API SERVER URL: http://localhost:8080


## Connect to the Qibo Service API

In [3]:
qibo_api = API(configuration=ConnectionConfiguration({
      "user_id": 4,
      "username": "qili-test-1",
      "api_key": "1e64261c-4e51-440b-8ae2-74eb36255a4b",
}))


[qibo-connection] 0.0.1|DEBUG|2021-12-13 14:52:24]: Connection successfully established.
[qibo-connection] 0.0.1|INFO|2021-12-13 14:52:24]: Storing personal qibo configuration...


## Load an already existing connection to the Qilimanjaro Service API

In [4]:
qili_api = API()

[qibo-connection] 0.0.1|DEBUG|2021-12-13 15:05:17]: Configuration loaded successfully.


## Checking if the connection is alive

In [5]:
qili_api.ping()

[qibo-connection] 0.0.1|DEBUG|2021-12-13 15:05:20]: Calling: http://localhost:8080/status


'OK'

## Listing all registered devices

In [6]:
devices = qili_api.list_devices()

[qibo-connection] 0.0.1|DEBUG|2021-12-13 15:05:23]: Calling: http://localhost:8080/api/v1/devices


In [6]:
devices

<Devices[2]:
{
  "device_id": 1,
  "device_name": "radagast-simulator",
  "status": "online",
  "characteristics": {
    "type": "simulator",
    "cpu": "Intel Core i9-9900K @ 16x 5GHz",
    "gpu": "NVIDIA GeForce RTX 3090",
    "os": "Ubuntu 20.04 focal",
    "kernel": "x86_64 Linux 5.4.0-80-generic",
    "ram": "64185MiB"
  }
}
{
  "device_id": 2,
  "device_name": "galadriel-quantum",
  "status": "online",
  "last_calibration_time": "Tue, 21 Sep 2021 07:58:57 GMT",
  "characteristics": {
    "type": "quantum"
  },
  "calibration_details": {
    "elapsed_time": 123,
    "t1": 12,
    "frequency": 988
  }
}

## Selecting a specific device

In [7]:
qili_api.select_device_id(device_id=1)

[qibo-connection] 0.0.1|INFO|2021-12-13 15:05:31]: Device 1 selected.


## Creating an experiment to be executed on a remote device

In [8]:
bellstate = AlgorithmDefinition(name=AlgorithmName.BELLSTATE,
                                type=AlgorithmType.GATE_BASED,
                                options=AlgorithmOptions(number_qubits=2,
                                                         initial_value=InitialValue.ONE))
experiment = ProgramDefinition(algorithms=[bellstate])

In [9]:
results = qili_api.execute(program=experiment)

[qibo-connection] 0.0.1|DEBUG|2021-12-13 15:05:36]: Sending experiment for a remote execution...
[qibo-connection] 0.0.1|DEBUG|2021-12-13 15:05:37]: Experiment completed successfully.


In [10]:
results[0]

array([1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j])